In [48]:
import pandas as pd 
import numpy as np
import sys
import re
pd.options.mode.chained_assignment = None

In [49]:
df = pd.read_table("all_table_definitions.txt", delimiter="|", encoding= 'unicode_escape')

In [50]:
df.rename(columns=lambda x: x.strip(), inplace=True)
new = df[df['Field Name'].str.contains('Table     :')]['Field Name'].str.split(' ', expand=True)[6]
index= new.index.values
df1= df.copy()
df1['Table Name'] = np.NaN
df1.loc[index, ['Table Name']] = new.values
df1['Table Name'].fillna(method='ffill', inplace=True)
df1 = df1[df1["Field Name"].str.contains("cmba|cmbb") == False]
df1.head()

,Field Name,Description,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,Table Name
0,,...,,length,,,Table Mode,NaN
1,---------------+------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 indt,...,tcdate,UTC Date/T 4,No,Yes,Not applicab,NaN
3,2 dsca,...,tcmcs.str50m,Multibyte 200,No,Yes,Not applicab,NaN
4,3 wfrc,...,asint.wfld,0,No,Yes,Not applicab,NaN


In [51]:
new1 = df1[df1['Field Name'].str.contains('Delete Reference Message')]['Field Name'].str.split(' ', expand=True)[8]
index1= new.index.values
df2= df1.copy()
df2['VCR'] = np.NaN
df2.loc[index1, ['VCR']] = new1.values
df2['VCR'].fillna(method='ffill', inplace=True)
df3 = df2[df2['VCR'] == '107U0']
df3.head()

,Field Name,Description,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,Table Name,VCR
22,Table : asint000 ...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
23,VRC : 107U0 a000 (Update) ...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
24,...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
25,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
26,Field Name,Description ...,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,asint000,107U0


In [52]:
table_dataset = pd.read_excel('Tables_name.xlsx')
table_dataset

,"project_id,""dataset_id"",""table_id"",""row_count"""
0,"s-bicloud-curated-0001-p,""ln_100"",""ttccom00110..."
1,"s-bicloud-curated-0001-p,""ln_100"",""ttcmcs00210..."
2,"s-bicloud-curated-0001-p,""ln_100"",""ttcmcs01010..."
3,"s-bicloud-curated-0001-p,""ln_100"",""ttcmcs01510..."
4,"s-bicloud-curated-0001-p,""ln_100"",""ttcmcs02310..."
...,...
1083,"s-bicloud-curated-0001-p,""ln_290"",""twhwmd21029..."
1084,"s-bicloud-curated-0001-p,""ln_290"",""twhwmd30029..."
1085,"s-bicloud-curated-0001-p,""ln_290"",""twhwmd30229..."
1086,"s-bicloud-curated-0001-p,""ln_290"",""twhwmd31029..."


In [53]:
table_dataset['project_id'] = table_dataset['project_id,"dataset_id","table_id","row_count"'].str.split(',', expand=True)[0]
table_dataset['dataset_id'] = table_dataset['project_id,"dataset_id","table_id","row_count"'].str.split(',', expand=True)[1]
table_dataset['table_id'] = table_dataset['project_id,"dataset_id","table_id","row_count"'].str.split(',', expand=True)[2]
table_dataset['row_count'] = table_dataset['project_id,"dataset_id","table_id","row_count"'].str.split(',', expand=True)[3]
table_dataset.drop('project_id,"dataset_id","table_id","row_count"', axis=1, inplace=True)
table_dataset['table_id'] = table_dataset['table_id'].str.replace('"', '')
table_dataset['dataset_id'] = table_dataset['dataset_id'].str.replace('"', '')
table_dataset['row_count'] = table_dataset['row_count'].str.replace('"', '').astype('int32')
table_dataset = table_dataset[table_dataset["dataset_id"].str.contains("_10_7") == False]
table_dataset

,project_id,dataset_id,table_id,row_count
0,s-bicloud-curated-0001-p,ln_100,ttccom001100,0
1,s-bicloud-curated-0001-p,ln_100,ttcmcs002100,6
2,s-bicloud-curated-0001-p,ln_100,ttcmcs010100,251
3,s-bicloud-curated-0001-p,ln_100,ttcmcs015100,0
4,s-bicloud-curated-0001-p,ln_100,ttcmcs023100,0
...,...,...,...,...
1083,s-bicloud-curated-0001-p,ln_290,twhwmd210290,0
1084,s-bicloud-curated-0001-p,ln_290,twhwmd300290,0
1085,s-bicloud-curated-0001-p,ln_290,twhwmd302290,0
1086,s-bicloud-curated-0001-p,ln_290,twhwmd310290,0


In [54]:
# table_dataset[table_dataset['table_id']=='tcisli305221']

In [55]:
# appended_data = []
# for name in ['cisli305']:
#     if name != 'cisli305':
#         newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
#         list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
#         newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
#         newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
#         nan_value = float("NaN")
#         newdf1.replace("", nan_value, inplace=True)
#         newdf1.dropna(subset = ["Field Name"], inplace=True)
#         newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
#         newdf1['Description'] = newdf1['Description'].map(lambda x: re.sub(r'[^a-zA-Z0-9 _]', '', x))
#         df6 = pd.DataFrame()
#         try: 
#             df4x = newdf1[newdf1['Datatype     Physic.'].str.contains(" x ")]
#             df4x['Field Name'] = df4x['Field Name'].str.strip()
#             df4x['Times'] = df4x['Datatype     Physic.'].str.split(' x ',  expand=True)[0].str.split(' ',  expand=True, n=2)[2]
#             df6 = df4x.loc[df4x.index.repeat(df4x.Times)]
#             cols = ['Field Name', 'Description', 'Domain', 'Datatype     Physic.', 'Mand.', 'Acti', 'Refer.   Reference', 'Table Name', 'VCR']
#             df6['Is_Dup'] = df6[cols].duplicated(keep=False).astype(int)
#             df6 = df6.sort_values(['Is_Dup'] + cols, ascending=[False] + [True] * len(cols))
#             df6['Dup_Ser'] = df6.groupby(cols).cumcount().add(1).mul(df6['Is_Dup'])
#             df6['Field Name'] = df6['Field Name'] + "_" + df6['Dup_Ser'].astype('str')
#             df6['Description'] = df6['Description'].str.split('(', expand=True, n=1)[0].str.strip()
#             df6['Description'] = df6['Description'] + "_" + df6['Dup_Ser'].astype('str')
#             df4withoutx = newdf1[~newdf1['Datatype     Physic.'].str.contains(" x ")]
#             data = df6.append(df4withoutx)
#             appended_data.append(data)
#             print("{} running".format(name))
#         except:
#             print("{} doesn't have duplicates".format(name))
#         finally:
#             df4withoutx = newdf1[~newdf1['Datatype     Physic.'].str.contains(" x ")]
#             data = df6.append(df4withoutx)
#             appended_data.append(data)
#     else:
#         newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
#         list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
#         newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
#         newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
#         nan_value = float("NaN")
#         newdf1.replace("", nan_value, inplace=True)
#         newdf1.dropna(subset = ["Field Name"], inplace=True)
#         newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
#         newdf1['Description'] = newdf1['Description'].map(lambda x: re.sub(r'[^a-zA-Z0-9 _]', '', x))
#         df6 = pd.DataFrame()
#         df4withoutx = newdf1[~newdf1['Datatype     Physic.'].str.contains(" x ")]
#         data = df6.append(df4withoutx)
#         appended_data.append(data)
# appended_data = pd.concat(appended_data, ignore_index=True)
# appended_data

In [56]:
# appended_data[appended_data['Table Name']=='cisli305']['Field Name'].unique()

In [57]:
# list_drop = 'cmbq|cmbw|cmbe|cmbr|cmbt|cmby|cmbu|cmbi|cmbo|cmbp|cmba|cmbs|cmbd|cmbf|cmbg|cmbh|cmbj|cmbj|cmbk|cmbl|cmbz|cmbx|cmbc|cmbv|cmbb|cmbn|cmbm|cmcq|cmcw|cmce|cmcr|cmct|cmcy|cmcu|cmci|cmco|cmcp|cmca|cmcs|cmcd|cmcf|cmcg|cmch|cmcj|cmck|cmcl|cmcz|cmcx|cmcc|cmcv|cmcb|cmcn|cmcm'
# appended_data[appended_data['Field Name'].str.strip().str.contains(list_drop)]

In [58]:
for dataset_id in table_dataset['dataset_id'].unique():
    table_dataset_new = table_dataset[table_dataset['dataset_id'] == dataset_id]
    table_names = list(table_dataset_new['table_id'])
    table_name = [short_name[1:] for short_name in table_names]
    table_name = [shorter_name[:-3] for shorter_name in table_name]
    appended_data = []
    table_data = []
    emty_data = []
    for name in table_name:
        if name != 'cisli305':
            try:
                newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
                list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
                newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
                newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
                nan_value = float("NaN")
                newdf1.replace("", nan_value, inplace=True)
                newdf1.dropna(subset = ["Field Name"], inplace=True)
                newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
                newdf1['Description'] = newdf1['Description'].map(lambda x: re.sub(r'[^a-zA-Z0-9 _]', '', x))
                df6 = pd.DataFrame()
                try: 
                    df4x = newdf1[newdf1['Datatype     Physic.'].str.contains(" x ")]
                    df4x['Field Name'] = df4x['Field Name'].str.strip()
                    df4x['Times'] = df4x['Datatype     Physic.'].str.split(' x ',  expand=True)[0].str.split(' ',  expand=True, n=2)[2]
                    df6 = df4x.loc[df4x.index.repeat(df4x.Times)]
                    cols = ['Field Name', 'Description', 'Domain', 'Datatype     Physic.', 'Mand.', 'Acti', 'Refer.   Reference', 'Table Name', 'VCR']
                    df6['Is_Dup'] = df6[cols].duplicated(keep=False).astype(int)
                    df6 = df6.sort_values(['Is_Dup'] + cols, ascending=[False] + [True] * len(cols))
                    df6['Dup_Ser'] = df6.groupby(cols).cumcount().add(1).mul(df6['Is_Dup'])
                    df6['Field Name'] = df6['Field Name'] + "_" + df6['Dup_Ser'].astype('str')
                    df6['Description'] = df6['Description'].str.split('(', expand=True, n=1)[0].str.strip()
                    df6['Description'] = df6['Description'] + "_" + df6['Dup_Ser'].astype('str')
                    df4withoutx = newdf1[~newdf1['Datatype     Physic.'].str.contains(" x ")]
                    data = df6.append(df4withoutx)
                    appended_data.append(data)
                except:
                    pass
                finally:
                    df4withoutx = newdf1[~newdf1['Datatype     Physic.'].str.contains(" x ")]
                    data = df6.append(df4withoutx)
                    appended_data.append(data)
                    table_data.append(name)
            except ValueError: 
                print("A ValueError occurred")
            except:
                emty_data.append(name)
        else:
            newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
            list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
            newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
            newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
            nan_value = float("NaN")
            newdf1.replace("", nan_value, inplace=True)
            newdf1.dropna(subset = ["Field Name"], inplace=True)
            newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
            newdf1['Description'] = newdf1['Description'].map(lambda x: re.sub(r'[^a-zA-Z0-9 _]', '', x))
            df6 = pd.DataFrame()
            df4withoutx = newdf1[~newdf1['Datatype     Physic.'].str.contains(" x ")]
            data = df6.append(df4withoutx)
            appended_data.append(data)
            table_data.append(name)
    appended_data = pd.concat(appended_data, ignore_index=True)
    list_drop = 'c001|c002|c003|c004|c005|c006|c007|c008|c009|cmbq|cmbw|cmbe|cmbr|cmbt|cmby|cmbu|cmbi|cmbo|cmbp|cmba|cmbs|cmbd|cmbf|cmbg|cmbh|cmbj|cmbj|cmbk|cmbl|cmbz|cmbx|cmbc|cmbv|cmbb|cmbn|cmbm|cmcq|cmcw|cmce|cmcr|cmct|cmcy|cmcu|cmci|cmco|cmcp|cmca|cmcs|cmcd|cmcf|cmcg|cmch|cmcj|cmck|cmcl|cmcz|cmcx|cmcc|cmcv|cmcb|cmcn|cmcm'
    appended_data = appended_data[~appended_data['Field Name'].str.strip().str.contains(list_drop)]
    appended_data.drop('Datatype     Physic.', axis=1, inplace=True)
    dic = {}
    original_stdout = sys.stdout # Save a reference to the original standard output
    ln_site = str(table_dataset_new['dataset_id'].str.split('ln_',expand=True)[1].values[0])
    with open('Query_ln_{}.txt'.format(ln_site), 'w') as f:
        for index, data in enumerate(table_data):
            databs = appended_data[['Field Name', 'Description','Table Name']]
            databs['Field Name'] = 'T_' + databs['Field Name'].astype(str).str.upper()
            newdata = databs[databs['Table Name'] == data]
            newdata['Description'] = newdata['Description'].str.replace(" ", "_")
            newdata['Name'] =  newdata['Field Name'] + ' AS '   + newdata['Description'].str.lower() + '_' +newdata['Field Name'].str.split('T_', expand=True)[1]
            newdata.drop_duplicates(subset=['Field Name', 'Description'], keep="first", inplace=True)
            names = list(newdata['Name'].str.replace('__', '_'))
            new_name =   ',\n'.join(names) 
            table_name = "t" + data
            source_dataset = 's-bicloud-curated-0001-p.ln_{}'.format(ln_site)
            destination_dataset = 's-finance-dev-0001-s.cons_LN'
            query_text_start = f"CREATE OR REPLACE VIEW `{destination_dataset}.{table_name}{ln_site}` AS \n"
            final_query = query_text_start + f"SELECT\n{new_name}\nFROM `{source_dataset}.{table_name}{ln_site}`;"
            dic[data] = final_query   
            sys.stdout = f # Change the standard output to the file we created.
            print(dic[data])
            sys.stdout = original_stdout # Reset the standard output to its original value

print('Create view successfully')


C:\Users\tandu\AppData\Local\Temp\ipykernel_17376\1902975096.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df6.append(df4withoutx)
C:\Users\tandu\AppData\Local\Temp\ipykernel_17376\1902975096.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df6.append(df4withoutx)
C:\Users\tandu\AppData\Local\Temp\ipykernel_17376\1902975096.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df6.append(df4withoutx)
C:\Users\tandu\AppData\Local\Temp\ipykernel_17376\1902975096.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df6.append(df4withoutx)
C:\Users\tandu\AppData\Local\Temp\ipykernel_17376\1902975096.py:

Create view successfully


In [59]:
# for dataset_id in table_dataset['dataset_id'].unique():
#     table_dataset_new = table_dataset[table_dataset['dataset_id'] == dataset_id]
#     table_names = list(table_dataset_new['table_id'])
#     table_name = [short_name[1:] for short_name in table_names]
#     table_name = [shorter_name[:-3] for shorter_name in table_name]
#     appended_data = []
#     table_data = []
#     emty_data = []
#     for name in table_name:
#         try:
#             # df4 = df3[df3['Datatype     Physic.'].str.contains(" x ") == True]
#             # df4['Field Name'] = df4['Field Name'].str.strip()
#             # df5 = df4[df4['Table Name'].isin(table_data)]
#             # df5['Times'] = df5['Datatype     Physic.'].str.split(' x ',  expand=True)[0].str.split(' ',  expand=True, n=2)[2]
#             # df6 = df5.loc[df5.index.repeat(df5.Times)]
#             # cols = ['Field Name', 'Description', 'Domain', 'Datatype     Physic.', 'Mand.', 'Acti', 'Refer.   Reference', 'Table Name', 'VCR']
#             # df6['Is_Dup'] = df6[cols].duplicated(keep=False).astype(int)
#             # df6 = df6.sort_values(['Is_Dup'] + cols, ascending=[False] + [True] * len(cols))
#             # df6['Dup_Ser'] = df6.groupby(cols).cumcount().add(1).mul(df6['Is_Dup'])
#             # df6['Field Name'] = df6['Field Name'] + "_" + df6['Dup_Ser'].astype('str')
#             # df6['Description'] = df6['Description'].str.split('(', expand=True, n=1)[0].str.strip()
#             # df6['Description'] = df6['Description'] + "_" + df6['Dup_Ser'].astype('str')
#             newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
#             list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
#             newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
#             newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
#             nan_value = float("NaN")
#             newdf1.replace("", nan_value, inplace=True)
#             newdf1.dropna(subset = ["Field Name"], inplace=True)
#             newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
#             #newdf1['Description'] = newdf1['Description'].str.replace('.','', regex=True).str.replace('/','_or_', regex=True).str.replace('-','', regex=True).str.replace('&','', regex=True).str.replace("'","", regex=True).str.replace("+","_plus_", regex=True).str.replace(",","", regex=True).str.replace(":","", regex=True).str.replace("%","", regex=True).str.replace("]","", regex=True).str.replace("[","", regex=True)
#             newdf1['Description'] = newdf1['Description'].map(lambda x: re.sub(r'[^a-zA-Z0-9 _]', '', x))
#             #newdf1['Datatype'] = newdf1['Datatype     Physic.'].str.split(' ',  expand=True, n=1)[1].str.split('  ',  expand=True, n=1)[0]
#             #newdf1['Physic.'] = newdf1['Datatype     Physic.'].str.split(' ',  expand=True, n=1)[1].str.split('  ',  expand=True, n=1)[1]
#             appended_data.append(newdf1)
#             table_data.append(name)
#         except ValueError: 
#             print("A ValueError occurred")
#         except:
#             emty_data.append(name)
#     appended_data = pd.concat(appended_data)
#     appended_data.drop('Datatype     Physic.', axis=1, inplace=True)
#     dic = {}
#     original_stdout = sys.stdout # Save a reference to the original standard output
#     ln_site = str(table_dataset_new['dataset_id'].str.split('ln_',expand=True)[1].values[0])
#     with open('Query_ln_{}.txt'.format(ln_site), 'w') as f:
#         for index, data in enumerate(table_data):
#             databs = appended_data[['Field Name', 'Description','Table Name']]
#             databs['Field Name'] = 'T_' + databs['Field Name'].astype(str).str.upper()
#             newdata = databs[databs['Table Name'] == data]
#             newdata['Description'] = newdata['Description'].str.replace(" ", "_")
#             newdata['Name'] =  newdata['Field Name'] + ' AS '   + newdata['Description'].str.lower() + '_' +newdata['Field Name'].str.split('T_', expand=True)[1]
#             newdata.drop_duplicates(subset=['Field Name', 'Description'], keep="first", inplace=True)
#             names = list(newdata['Name'].str.replace('__', '_'))
#             new_name =   ',\n'.join(names) 
#             table_name = "t" + data
#             source_dataset = 's-bicloud-curated-0001-p.ln_{}'.format(ln_site)
#             destination_dataset = 's-finance-dev-0001-s.cons_LN'
#             query_text_start = f"CREATE OR REPLACE VIEW `{destination_dataset}.{table_name}{ln_site}` AS \n"
#             final_query = query_text_start + f"SELECT\n{new_name}\nFROM `{source_dataset}.{table_name}{ln_site}`;"
#             dic[data] = final_query   
#             sys.stdout = f # Change the standard output to the file we created.
#             print(dic[data])
#             sys.stdout = original_stdout # Reset the standard output to its original value
# print('Create view successfully')
